In [1]:
from py2neo import Graph
graph = Graph("bolt: // localhost:7687", auth=("neo4j", "12345678"))

In [2]:
from py2neo import NodeMatcher
# 初始化一个 matcher 实例
matcher = NodeMatcher(graph)
# 用 match 方法查找，返回一个 NodeMatch 对象
result = matcher.match("明星", name="周杰伦")
# first 方法返回查询结果的第一个
result.first()
# 通过 list 来把所有结果显示出来
list(result)

[Node('明星', name='周杰伦', 作品=' 叱咤风云 《功夫熊猫3》音... 惊天魔盗团2 天台爱情 神探·李奥 逆战', 生日='出生日期：\r\n                                                1979-01-18                    ', 职业='职业：\r\n                                                导演                            \xa0\xa0演员                            \xa0\xa0歌手                                                ')]

In [4]:
# cypher 语句，查询并返回所有的节点
query = "match (n) return n"
# 执行 cypher 语句，获得返回结果
cursors = graph.run(query)
# 通过遍历的方式取出所有结果
for i in cursors:
    print(i)

Node('明星', name='文章', 作品=' 测谎人 胖子行动队 封神传奇 过年好 陆垚知马俐 王朝的女人·杨贵...', 生日='出生日期：\r\n                                                1984-06-26                    ', 职业='职业：\r\n                                                导演                            \xa0\xa0演员                            \xa0\xa0编剧                                                ')
Node('明星', name='乔任梁', 作品=' 我们的十年 魔卡行动 万万没想到 锦绣缘华丽冒险 疯狂72小时 小时代', 生日='出生日期：\r\n                                                1987-10-15                    ', 职业='职业：\r\n                                                演员                            \xa0\xa0歌手                                                ')
Node('明星', name='张博', 作品=' 危机先生 疯狂熊孩子 王大花的革命生涯 王大花的革命生涯 金战 萧红', 生日='出生日期：\r\n                                                1982-08-29                    ', 职业='职业：\r\n                                                演员                                                ')
Node('明星', name='明道', 作品=' 市长夫人的秘密 阿修罗 绑架者 恋爱教父之三个“... 咱们结婚吧 私房钱', 生日='出

# 对节点和关系的增删改查功能封装成函数并综合在类中

In [96]:
from py2neo import Graph, Node, Relationship
from py2neo.matching import NodeMatcher

class Database:
    def __init__(self, uri, username, password):
        self.graph = Graph(uri, auth=(username, password))

    #创建节点函数
    def create_node(self, label, name, birthday=None, occupation=None, works=None):
        properties = {"name": name}
        if birthday is not None:
            properties["生日"] = birthday
        if occupation is not None:
            properties["职业"] = occupation
        if works is not None:
            properties["作品"] = works

        node = Node(label, **properties)
        self.graph.create(node)
        return node
    
    #删除节点函数
    def delete_node(self, label, properties):
        query = f"MATCH (n:{label} {properties}) DELETE n"
        result = self.graph.run(query)
        print("节点删除成功")

    #更新节点函数
    def update_node(self, label, name, properties):
        updates = [f"n.{key} = '{value}'" for key, value in properties.items()]
        query = f"MATCH (n:{label} {{name: '{name}'}}) SET {', '.join(updates)} RETURN n"
        result = self.graph.evaluate(query)
        if result:
            return result
        else:
            return None

    #查询节点函数
    def get_nodes(self, label, **kwargs):
        matcher = NodeMatcher(self.graph)
        return matcher.match(label, **kwargs)

    #创建关系函数
    def create_relationship(self, star_name1, rel_type, star_name2, properties):
        nodes = self.get_nodes("明星")
        start_node = None
        end_node = None
        for node in nodes:
            if node["name"] == star_name1:
                start_node = node
            elif node["name"] == star_name2:
                end_node = node

        if start_node and end_node:
            relationship = Relationship(start_node, rel_type, end_node, **properties)
            self.graph.create(relationship)
            print("关系创建成功")
            return relationship
        else:
            print("未找到对应的节点")
            return None

    #删除关系函数
    def delete_relationship(self, start_name, rel_type, end_name):
        query = f"MATCH (s1)-[r:{rel_type}]->(s2) " \
                f"WHERE s1.name = '{start_name}' AND s2.name = '{end_name}' " \
                f"DELETE r"
        result = self.graph.run(query)
        deleted_relationship = result.evaluate()
        print("关系删除成功")

    #更新关系函数
    def update_relationship(self, star_name1, old_rel_type, new_rel_type, star_name2):
        query = f"MATCH (start:明星 {{name: '{star_name1}'}})-[rel:{old_rel_type}]->(end:明星 {{name: '{star_name2}'}}) " \
                f"DELETE rel " \
                f"CREATE (start)-[new_rel:{new_rel_type}]->(end) " \
                f"RETURN new_rel"
        result = self.graph.run(query)
        updated_relationship = result.data()
        if updated_relationship:
            print("关系更新成功")
        else:
            print("关系更新失败")

        return updated_relationship

    #查询关系函数
    def get_relationships(self, start_name, rel_type, end_name):
        query = f"MATCH (start:明星 {{name: '{start_name}'}})-[r:{rel_type}]->(end:明星 {{name: '{end_name}'}}) RETURN r"
        result = self.graph.run(query)
        relationships = [record["r"] for record in result]

        if relationships:
            for rel in relationships:
                print(rel)
            print("查询成功")
        else:
            print("不存在该关系")
        
        return relationships

# 实例化类并测试具体函数

In [97]:
# 实例化Database类，指定数据库连接信息
db = Database("bolt://localhost:7687", "neo4j", "12345678")

## 查询节点与关系：

In [7]:
# 获取节点
nodes = db.get_nodes("明星", name="周星驰")
for node in nodes:
    print(node)

(_595:明星 {name: '\u5468\u661f\u9a70', 作品: ' \u529f\u592b2 \u592a\u6781 \u7f8e\u4eba\u9c7c2 \u300a\u65b0\u559c\u5267\u4e4b\u738b\u300b\u5317... \u300a\u65b0\u559c\u5267\u4e4b\u738b\u300b\u201c... \u65b0\u559c\u5267\u4e4b\u738b', 生日: '\u51fa\u751f\u65e5\u671f\uff1a\r\n                                                1962-06-22                    ', 职业: '\u804c\u4e1a\uff1a\r\n                                                \u5bfc\u6f14                            \u00a0\u00a0\u6f14\u5458                            \u00a0\u00a0\u7f16\u5267                            \u00a0\u00a0\u5236\u4f5c\u4eba                                                '})


In [8]:
# 获取关系，输入关系名称
db.get_relationships("张博", "拍档", "何润东")

(张博)-[:拍档 {}]->(何润东)
查询成功


[拍档(Node('明星', name='张博', 作品=' 危机先生 疯狂熊孩子 王大花的革命生涯 王大花的革命生涯 金战 萧红', 生日='出生日期：\r\n                                                1982-08-29                    ', 职业='职业：\r\n                                                演员                                                '), Node('明星', name='何润东', 作品=' 浣溪沙 征途 八子 游戏规则 三少爷的剑 蜜月酒店杀人事件', 生日='出生日期：\r\n                                                1979-09-13                    ', 职业='职业：\r\n                                                演员                            \xa0\xa0歌手                                                '))]

## 增加节点与关系：

In [18]:
# 创建节点
node = db.create_node("明星", "测试2", birthday="1956-09-12", occupation="演员", works=["测试电影", "movie1"])
print(node)

(_990:明星 {name: '\u6d4b\u8bd52', 作品: ['\u6d4b\u8bd5\u7535\u5f71', 'movie1'], 生日: '1956-09-12', 职业: '\u6f14\u5458'})


In [35]:
# 创建关系 完成
properties = {"relation": "好友"}
db.create_relationship("测试1", "好友", "测试2", properties)

关系创建成功


好友(Node('明星', birthday='1956-09-12', name='测试1', occupation='演员', works=['测试电影', 'movie1']), Node('明星', name='测试2', 作品=['测试电影', 'movie1'], 生日='1956-09-12', 职业='演员'), relation='好友')

## 更新节点与关系：

In [49]:
# 更新节点
node_properties = {"生日": "1975-01-01", "职业": "Actor", "工作": "Movie A, Movie B"}
updated_node = db.update_node("明星", "测试1", node_properties)

if updated_node:
    # 打印更新后的节点信息
    print(updated_node)
else:
    print("节点更新失败")

(_986:明星 {birthday: '1956-09-12', name: '\u6d4b\u8bd51', occupation: '\u6f14\u5458', works: ['\u6d4b\u8bd5\u7535\u5f71', 'movie1'], 工作: 'Movie A, Movie B', 生日: '1975-01-01', 职业: 'Actor'})


In [81]:
# 更新关系
updated_relationship = db.update_relationship("测试1", "兄弟", "旧情", "测试2")

if updated_relationship:
    print(updated_relationship)
else:
    print("关系更新失败")

关系更新成功
[{'new_rel': 旧情(Node('明星', birthday='1956-09-12', name='测试1', occupation='演员', works=['测试电影', 'movie1'], 工作='Movie A, Movie B', 生日='1975-01-01', 职业='Actor'), Node('明星', name='测试2', 作品=['测试电影', 'movie1'], 生日='1956-09-12', 职业='演员'))}]


## 删除节点与关系：

In [99]:
# 删除节点
properties = "{name: '测试2'}"
db.delete_node("明星", properties)

节点删除成功


In [98]:
# 删除关系
db.delete_relationship("测试1", "旧情", "测试2")

关系删除成功
